In [45]:
! pip install sklearn altair

In [32]:
import pandas as pd
import altair as alt

from py2neo import Graph
from sklearn.manifold import TSNE

In [2]:
graph = Graph('bolt://100.27.6.130:7687', auth=('neo4j', 'blanket-cam-reinforcement'))

In [48]:
query = """
MATCH (n)
DETACH DELETE n;
"""

graph.run(query)

(No data)

In [54]:
query = """
CALL gds.graph.drop('E-Commerce')
"""

graph.run(query)

graphName,database,memoryUsage,sizeInBytes,nodeProjection,relationshipProjection,nodeQuery,relationshipQuery,nodeCount,relationshipCount,nodeFilter,relationshipFilter,density,creationTime,modificationTime,schema
E-Commerce,neo4j,,-1,"{Customer: {properties: {}, label: 'Customer'}, Product: {properties: {}, label: 'Product'}}","{ORDERS: {orientation: 'UNDIRECTED', aggregation: 'DEFAULT', type: 'ORDERS', properties: {quantity: {property: 'quantity', aggregation: 'DEFAULT', defaultValue: null}}}}",null,null,0,0,null,null,0.0,datetime('2022-01-10T20:40:56.926006000+00:00'),datetime('2022-01-10T20:40:56.936641000+00:00'),"{relationships: {ORDERS: {quantity: 'Float (DefaultValue(NaN), PERSISTENT, Aggregation.DEFAULT)'}}, nodes: {Customer: {}, Product: {}}}"


In [50]:
query = """
CALL apoc.periodic.iterate(
"CALL apoc.load.csv('https://raw.githubusercontent.com/BiaPri/Graph_Field/master/data_e_commerce/customers.csv')
 YIELD map AS row RETURN row",
"MERGE (c:Customer {id: toInteger(row.customer_id), name: row.customer_name, gender: row.gender, age: toInteger(row.age)})
 MERGE (ct:City {city_name: row.city, zip_code: toInteger(row.zip_code), address: row.home_address})
 MERGE (s:State {name: row.state})
 MERGE (c)-[:LIVING_CITY]->(ct)
 MERGE (c)-[:LIVING_STATE]->(s)",
 {batchSize: 150}
)
"""

graph.run(query)

batches,total,timeTaken,committedOperations,failedOperations,failedBatches,retries,errorMessages,batch,operations,wasTerminated,failedParams,updateStatistics
7,1000,1,1000,0,0,0,{},"{total: 7, committed: 7, failed: 0, errors: {}}","{total: 1000, committed: 1000, failed: 0, errors: {}}",false,{},"{nodesDeleted: 0, labelsAdded: 2008, relationshipsCreated: 2000, nodesCreated: 2008, propertiesSet: 7008, relationshipsDeleted: 0, labelsRemoved: 0}"


In [51]:
query = """
CALL apoc.periodic.iterate(
"CALL apoc.load.csv('https://raw.githubusercontent.com/BiaPri/Graph_Field/master/data_e_commerce/products.csv')
 YIELD map AS row RETURN row",
"MERGE (:Product {id: toInteger(row.product_ID), type: row.product_type, name: row.product_name, 
                 size: row.size, color: row.colour, price: toInteger(row.price), stock: toInteger(row.quantity)})",
 {batchSize: 200}
)
"""

graph.run(query)

batches,total,timeTaken,committedOperations,failedOperations,failedBatches,retries,errorMessages,batch,operations,wasTerminated,failedParams,updateStatistics
7,1260,1,1260,0,0,0,{},"{total: 7, committed: 7, failed: 0, errors: {}}","{total: 1260, committed: 1260, failed: 0, errors: {}}",false,{},"{nodesDeleted: 0, labelsAdded: 1260, relationshipsCreated: 0, nodesCreated: 1260, propertiesSet: 8820, relationshipsDeleted: 0, labelsRemoved: 0}"


In [52]:
query = """
CALL apoc.periodic.iterate(
"CALL apoc.load.csv('https://raw.githubusercontent.com/BiaPri/Graph_Field/master/data_e_commerce/orders_sales.csv')
 YIELD map AS row RETURN row",
"MATCH (c:Customer {id: toInteger(row.customer_id)}) 
 MATCH (p:Product {id: toInteger(row.product_id)})
 MERGE (c)-[:ORDERS {order_date: date(row.order_date), delivery_date: date(row.delivery_date), quantity: toInteger(row.quantity)}]->(p)",
 {batchSize: 500}
)
"""

graph.run(query)

batches,total,timeTaken,committedOperations,failedOperations,failedBatches,retries,errorMessages,batch,operations,wasTerminated,failedParams,updateStatistics
10,5000,14,5000,0,0,0,{},"{total: 10, committed: 10, failed: 0, errors: {}}","{total: 5000, committed: 5000, failed: 0, errors: {}}",false,{},"{nodesDeleted: 0, labelsAdded: 0, relationshipsCreated: 4999, nodesCreated: 0, propertiesSet: 14997, relationshipsDeleted: 0, labelsRemoved: 0}"


In [55]:
query = """
CALL gds.graph.create('E-Commerce', ['Customer', 'Product'],
                     {
                        ORDERS: {
                                  type: 'ORDERS',
                                  orientation: 'UNDIRECTED',
                                  properties: 'quantity'
                                 }
                           }
                       );
"""

graph.run(query)

nodeProjection,relationshipProjection,graphName,nodeCount,relationshipCount,createMillis
"{Customer: {properties: {}, label: 'Customer'}, Product: {properties: {}, label: 'Product'}}","{ORDERS: {orientation: 'UNDIRECTED', aggregation: 'DEFAULT', type: 'ORDERS', properties: {quantity: {property: 'quantity', aggregation: 'DEFAULT', defaultValue: null}}}}",E-Commerce,2260,9998,179


In [56]:
query = """
CALL gds.beta.node2vec.write('E-Commerce',
                             {
                               embeddingDimension: 25,
                               iterations: 10,
                               walkLength: 10,
                               writeProperty: "embeddingNode2vec"
                             }
                           );
"""

graph.run(query)

nodeCount,nodePropertiesWritten,createMillis,computeMillis,writeMillis,configuration
2260,2260,0,13108,185,"{negativeSamplingExponent: 0.75, initialLearningRate: 0.025, writeConcurrency: 4, negativeSamplingRate: 5, walksPerNode: 10, positiveSamplingFactor: 0.001, relationshipWeightProperty: null, iterations: 10, returnFactor: 1.0, concurrency: 4, walkLength: 10, windowSize: 10, writeProperty: 'embeddingNode2vec', inOutFactor: 1.0, embeddingDimension: 25, nodeLabels: ['*'], sudo: false, minLearningRate: 0.0001, relationshipTypes: ['*'], walkBufferSize: 1000, username: null}"


In [57]:
query = """
MATCH (c:Customer)
RETURN c.name as customer, c.embeddingNode2vec as embedding
"""

df = graph.run(query).to_data_frame() #Better by adding the other parameters
df.head()

,customer,embedding
0,Leanna Busson,"[0.47761070728302, 0.6816880106925964, 1.97443..."
1,Zabrina Harrowsmith,"[-0.9500146508216858, 0.17025096714496613, 0.4..."
2,Shina Dullaghan,"[-0.9062536954879761, 0.203145369887352, 0.574..."
3,Hewet McVitie,"[0.5305970311164856, -0.8383355736732483, -0.2..."
4,Rubia Ashleigh,"[0.8262993693351746, 0.24164162576198578, 0.95..."


In [58]:
X_embedded = TSNE(n_components=2, random_state=6).fit_transform(list(df.embedding))

customer = df.customer
df = pd.DataFrame(data = {
    "customer": customer,
    "x": [value[0] for value in X_embedded],
    "y": [value[1] for value in X_embedded]
})

C:\Users\BPRN\Anaconda3\envs\neo4j\lib\site-packages\sklearn\manifold\_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
C:\Users\BPRN\Anaconda3\envs\neo4j\lib\site-packages\sklearn\manifold\_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,


In [59]:
alt.Chart(df).mark_circle(size=60).encode(
    x='x',
    y='y',
).properties(width=700, height=400)

alt.Chart(...)